# Create Descriptor Notebook
This notebook contains helper functions for creating the notebook library descriptor file other notebooks can use to enable Compose Functionality.

Note, there are still some issues with the Kotlin Jupyter Kernel. Right now, it looks like it might need to contain Compose Skia wrappers for all platforms we want to support

## How to Use?

1. Run the entire notebook
2. Copy the output of the last cell into a `compose-dev.json` file at the root of this project.
3. Use it inside you project like this `%use compose@file[/Users/christian.melchior/Private/compose-notebook/compose-dev.json]`. Adjust the path as needed.


In [ ]:
// No data inserted here when CMD + C -> CMD + V

In [5]:
val fileTemplate = """
{
  "description": "Compose Support for Notebooks",
  "properties": [
    { "name": "compose", "value": "1.7.3" },
    { "name": "platform", "value": "macos-arm64" },
    { "name": "coroutines", "value": "1.8.0" },
    { "name": "skiko", "value": "0.8.18" },
    { "name": "androidx-collection", "value": "1.4.5" },
    { "name": "lifecycle-common", "value": "2.8.7" },
    { "name": "lifecycle", "value": "2.8.4" },
    { "name": "v-renovate-hint", "value": "update: package=org.jetbrains.kotlinx:dataframe" }
  ],
  "link": "https://github.com/Kotlin/kotlin-jupyter",
  "init": [
    <formattedInitCode>
  ],
  "repositories": [
    "https://maven.google.com"
  ],
  "dependencies": [
    "org.jetbrains.compose:compose-full:${'$'}compose",
    "org.jetbrains.kotlinx:kotlinx-coroutines-core-jvm:${'$'}coroutines",
    "org.jetbrains.skiko:skiko-awt-runtime-${'$'}platform:${'$'}skiko",
    "org.jetbrains.skiko:skiko-awt:${'$'}skiko",
    "androidx.collection:collection-jvm:${'$'}androidx-collection",
    "androidx.lifecycle:lifecycle-common:${'$'}lifecycle-common",
    "androidx.lifecycle:lifecycle-viewmodel-compose-desktop:${'$'}lifecycle",
    "androidx.lifecycle:lifecycle-runtime-compose-desktop:${'$'}lifecycle",
    "org.jetbrains.compose.ui:ui-test-junit4-desktop:${'$'}compose",
    "org.jetbrains.compose.ui:ui-graphics-desktop:${'$'}compose",
    "org.jetbrains.kotlinx:kotlinx-coroutines-test:${'$'}coroutines"
  ],
  "imports": [
    "androidx.compose.ui.test.ExperimentalTestApi",
    "androidx.compose.ui.test.runDesktopComposeUiTest",
    "androidx.compose.ui.graphics.*",
    "androidx.compose.ui.*",
    "androidx.compose.ui.draw.*",
    "androidx.compose.ui.text.style.*",
    "androidx.compose.foundation.*",
    "androidx.compose.foundation.layout.*",
    "androidx.compose.foundation.shape.*",
    "androidx.compose.material.*",
    "androidx.compose.runtime.*",
    "androidx.compose.ui.unit.*",
    "androidx.compose.ui.window.*",
    "androidx.compose.foundation.shape.*",
    "androidx.compose.ui.awt.ComposePanel",
    "org.jetbrains.skia.Bitmap",
    "org.jetbrains.skiko.toBufferedImage",
    "java.awt.BorderLayout",
    "java.awt.Dimension",
    "java.awt.image.BufferedImage",
    "javax.swing.JComponent",
    "javax.swing.JPanel",
    "javax.swing.SwingUtilities"
  ]
}
""".trimIndent()

In [2]:
val initCode = """
@file:OptIn(ExperimentalTestApi::class)
import java.awt.BorderLayout
import java.awt.event.MouseWheelEvent
import javax.swing.JPanel
import java.awt.LayoutManager

class NonPropagatingPanel(layout: LayoutManager) : JPanel(layout) {
  override fun processMouseWheelEvent(e: MouseWheelEvent) {
    if (shouldConsumeScroll(e)) {
      e.consume()
    } else {
      super.processMouseWheelEvent(e)
    }
  }
  private fun shouldConsumeScroll(e: MouseWheelEvent): Boolean {
    return true // Prevent scroll events from leaking outside the Compose Panel
  }
}


fun composeScreenshot(
    width: Int = 1024,
    height: Int = 768,
    backgroundColor: androidx.compose.ui.graphics.Color = androidx.compose.ui.graphics.Color.White,
    content: @Composable () -> Unit,
): BufferedImage {
    lateinit var image: BufferedImage
    runDesktopComposeUiTest(width, height) {
        setContent(composeContent(backgroundColor, content))
        val screenshot: Bitmap = this.captureToImage().asSkiaBitmap()
        image = screenshot.toBufferedImage()
    }
    return image
}

private fun composeContent(
    backgroundColor: androidx.compose.ui.graphics.Color,
    content: @Composable () -> Unit
): @Composable () -> Unit {
    val wrappedContent: @Composable () -> Unit = {
        Box(
            modifier = Modifier.fillMaxSize().background(backgroundColor),
            contentAlignment = Alignment.Center
        ) {
            content()
        }
    }
    return wrappedContent
}

fun COMPOSE(
    width: Int = 1024,
    height: Int = 300,
    backgroundColor: androidx.compose.ui.graphics.Color = androidx.compose.ui.graphics.Color.Transparent,
    content: @Composable () -> Unit
): Any {
    if (notebook.kernelRunMode.inMemoryOutputsSupported) {
        val screenWidth = width // Will auto-expand, but must be > 0
        val screenHeight = height // Default starting height. If Compose view is higher it will auto-expand.
        val mainPanel = NonPropagatingPanel(BorderLayout())
        mainPanel.isOpaque = true
        mainPanel.preferredSize = Dimension(screenWidth, screenHeight)
        mainPanel.size = Dimension(screenWidth, screenHeight)
        mainPanel.isVisible = true
        var composePanel: JComponent? = null
        SwingUtilities.invokeAndWait {
            val panel = ComposePanel()
            panel.isOpaque = false
            panel.setContent(composeContent(backgroundColor, content))
            composePanel = panel
        }
        mainPanel.add(composePanel!!, BorderLayout.CENTER)
        return mainPanel
    } else {
        val image = composeScreenshot(
            width,
            height,
            backgroundColor,
            content,
        )
        return image
    }
}
""".trimIndent()

In [3]:
val formattedInitCode = "\"${initCode.replace("\"", "\\\"").replace("\n", "\\n")}\""

In [4]:
fileTemplate.replace("<formattedInitCode>", formattedInitCode)

{
  "description": "Compose Support for Notebooks",
  "properties": [
    { "name": "compose", "value": "1.7.3" },
    { "name": "platform", "value": "macos-arm64" },
    { "name": "coroutines", "value": "1.8.0" },
    { "name": "skiko", "value": "0.8.18" },
    { "name": "androidx-collection", "value": "1.4.5" },
    { "name": "lifecycle-common", "value": "2.8.7" },
    { "name": "lifecycle", "value": "2.8.4" },
    { "name": "v-renovate-hint", "value": "update: package=org.jetbrains.kotlinx:dataframe" }
  ],
  "link": "https://github.com/Kotlin/kotlin-jupyter",
  "init": [
    "@file:OptIn(ExperimentalTestApi::class)\nimport java.awt.BorderLayout\nimport java.awt.event.MouseWheelEvent\nimport javax.swing.JPanel\nimport java.awt.LayoutManager\n\nclass NonPropagatingPanel(layout: LayoutManager) : JPanel(layout) {\n  override fun processMouseWheelEvent(e: MouseWheelEvent) {\n    if (shouldConsumeScroll(e)) {\n      e.consume()\n    } else {\n      super.processMouseWheelEvent(e)\n    }\